In [13]:

#

#& Loading & Exploring the dataset


#Import Libraries
import numpy as np
import pandas as pd

# Load data

file = pd.read_csv('Orders_with_issues.csv')

# print(file)
df = pd.DataFrame(file)

# print(df)


# print(file)

# Preview
#print("🔍 Sample Data:")
# file.head()

# Summary
#print("\n🧾 Column Info:")
# file.info()

print('\n')

# Check missing values
#print("\n🚨 Missing Values:")
# file.isna().sum()


# Check unique shipping companies

uniqueShipComp = df['ShippingCompany'].unique()   # or: file['ShippingCompany'].unique()

print(uniqueShipComp)

print('\n')
# check no of shipping order for each of shipping companies

noUniqueComp=df['ShippingCompany'].value_counts()
print(noUniqueComp)




['Kiwilytics Goods Shipping LLC.' 'UPS Worldwide' 'FedEx Logistics'
 'Aramex International' 'DHL Express']


ShippingCompany
FedEx Logistics                   54
DHL Express                       53
Aramex International              51
UPS Worldwide                     46
Kiwilytics Goods Shipping LLC.    41
Name: count, dtype: int64


In [14]:
#
import numpy as np
import pandas as pd

# & Cleaning Dates, numeric entries that don't make sense


# Convert dates (Error Types: #coerce # ignore #raise)


df["OrderDate"] = pd.to_datetime(df["OrderDate"], errors="coerce")
# print(df['OrderDate'].dtype)
# print('\n')

df["ShippedDate"] = pd.to_datetime(df["ShippedDate"], errors="coerce")
# print(df['ShippedDate'].dtype)

# print('\n')

# print(df.head())

# pd.to_datetime() → converts the column into proper datetime objects.

# errors='coerce' → replaces invalid entries like "not_a_date" with NaT (null datetime).

# Clean shipping cost

df["ShippingCost"] = pd.to_numeric(df["ShippingCost"], errors="coerce")
print(df["ShippingCost"].dtype)

df.loc[df["ShippingCost"] < 1, "ShippingCost"] = np.nan
print("\n")
# print(df.head())

df['ShippingCost']=df['ShippingCost'].fillna(df['ShippingCost'].median())

#* to verify:
print(df['ShippingCost'].isna().sum())  #! should be zero



#^ Drop rows where both dates are missing

df = df[~(df['OrderDate'].isna() & df['ShippedDate'].isna())]

#* df['OrderDate'].isna() → True where OrderDate is missing.

#* df['ShippedDate'].isna() → True where ShippedDate is missing.

# & → logical AND → True only where both are missing.

# ~(...) → flips it → True where NOT both missing.

#* df[...] → keeps only rows where the condition is True.

#^ ✅ Meaning in plain words
#* This line removes rows where both OrderDate and ShippedDate are missing.
#* If at least one of them is present, the row is kept.
#* If both are NaN, the row is dropped.

print(len(df))   #! “How many rows are in my DataFrame?”

float64


0
242


In [ ]:
#
import numpy as np
import pandas as pd
#& Handling Nulls, Fixing Fields & Standardization

df = pd.read_csv('Orders_with_issues.csv')
print(df.head())

# Handle nulls
df['OrderID']=df['OrderID'].fillna(method='ffill')
#? or: because method ='ffill' is deprecated
df['OrderID']=df['OrderID'].ffill()

# print(df['OrderID'].isna().sum())
print('\n')
df['CustomerID']=df['CustomerID'].fillna('unknown')
# print(df['CustomerID'].isna().sum())

print('\n')
df['ShipCity']=df['ShipCity'].fillna('unspecified')
# print(df['ShipCity'].isna().sum())

# Standardize names
df['ShipCountry'] = df['ShipCountry'].str.strip()
df['ShipCountry'] = df['ShipCountry'].str.title()

df['ShipCity'] = df['ShipCity'].str.strip()
df['ShipCity'] = df['ShipCity'].str.title()

df['ShippingCompany'] = df['ShippingCompany'].str.strip()

# print(df.head())


# Fix specific cases

df.loc[df['ShippingCompany'].str.contains("UPS", na=False), 'ShippingCompany'] = 'ups Limited'
print(df.head())



   OrderID CustomerID   OrderDate ShippedDate        ShippingCost ShipCountry  \
0   1000.0       C001  2025-05-17  2025-07-30  -57.25454602444980     Germany   
1   1001.0       C002  2025-01-26  2025-07-30              320.61      Canada   
2   1002.0       C003  2025-03-08  2025-07-30              165.17      Canada   
3   1003.0       C004  2025-03-24  2025-07-30               12.55     Germany   
4   1004.0       C005  2025-04-15  2025-07-30              186.36      Canada   

    ShipCity                 ShippingCompany  
0    Hamburg  Kiwilytics Goods Shipping LLC.  
1   Montreal                   UPS Worldwide  
2  Vancouver                 FedEx Logistics  
3     Munich            Aramex International  
4  Vancouver                 FedEx Logistics  
   OrderID CustomerID   OrderDate ShippedDate        ShippingCost ShipCountry  \
0   1000.0       C001  2025-05-17  2025-07-30  -57.25454602444980     Germany   
1   1001.0       C002  2025-01-26  2025-07-30              320.61    

In [16]:
#
import numpy as np
import pandas as pd

# &  Feature Engineering
# Days between order and shipment
df["deliveryTime"] = df["ShippedDate"] - df["OrderDate"]
df['DeliveryDays'] = (df['ShippedDate'] - df['OrderDate']).dt.days  # will be in float

# deliveryTime → full timedelta object (keeps precision, can show “12 days 05:30:00”).

# DeliveryDays → just the numerical (float) count of days (no hours/minutes).

# df.head()

# Flags
def isOnTime(x):
    if pd.isna(x):
        return 'unknown'
    if x > 15:
        return 'late'
    else:
        return 'onTime'
    
df['deliveryStatus'] = df['DeliveryDays'].apply(isOnTime)

# df.tail()

# Domestic vs International

domestic_countries = ['Germany']

def isDomestic(x):
   if pd.isna(x):
     return 'unknown'
   if x in domestic_countries:
     return 'Domestic'
   else:
     return 'international'

df['Shipment location'] = df['ShipCountry'].apply(isDomestic)


# df.head()

print(df.groupby('CustomerID')['OrderID'].count().head())

def isRegular(x):
    if pd.isna(x):
     return 'Unknown'
    if x >=9:
     return 'Regular Customer'
    else:
     return 'Normal Customer'
 
df['Customer Loyalty'] = df.groupby('CustomerID')['OrderID'].transform('count').apply(isRegular)
print('\n')
df.head()

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
# Grouping examples
import numpy as np
import pandas as pd

avg_ship_cost= df.groupby('ShippingCompany')['ShippingCost'].median()
print(avg_ship_cost)



ShippingCompany
Aramex International              234.35
DHL Express                       234.22
FedEx Logistics                   216.93
Kiwilytics Goods Shipping LLC.    234.22
UPS Limited                       234.22
Name: ShippingCost, dtype: float64


In [ ]:

#
import numpy as np
import pandas as pd

#&  Export, Reporting & Wrap-up
# Export cleaned file


# Final summary

late_orders = df[df['deliveryStatus']=='late'].shape[0]
print(late_orders)
# orders = df.groupby['deliveryStatus']
print('\n')

# print(df['deliveryStatus'].value_counts())

orders_per_country= df.groupby('ShipCountry')['OrderID'].count()
# print(orders_per_country)
# print('\n')
orders_per_city= df.groupby('ShipCity')['OrderID'].count()
# print(orders_per_city)
# print('\n')

# frequency of orders handled by each shipping company
no_ship_comp=df['ShippingCompany'].value_counts().head(3)
print(no_ship_comp)  # market share of shipping companies in your dataset
# df.head()

190


ShippingCompany
FedEx Logistics                   53
DHL Express                       52
Aramex International              51
UPS Limited                       45
Kiwilytics Goods Shipping LLC.    41
Name: count, dtype: int64
